In [2]:
import pandas as pd

from eeg_snn_encoder.config import REPORTS_DIR

encoder_code = ["poisson", "tbr", "sf", "bsa", "pe", "be", "dummy"]

2025-06-13 22:28:21.227 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: E:\Projects\snn-encoder-test


In [8]:
import json

params_list = []

for code in encoder_code:
    with open(REPORTS_DIR / f'{code}_model_params_new.json', 'r') as file:
        raw_data = json.load(file)
        params_list.append(raw_data["optimizer_params"] | raw_data["model_params"] | { "encoder_params": raw_data["encoder_params"] })


params_df = pd.DataFrame(params_list, index=encoder_code).T
params_df

,poisson,tbr,sf,bsa,pe,be,dummy
lr,0.000042,0.000075,0.000087,0.000068,0.000094,0.000039,0.000045
weight_decay,0.000014,0.000002,0.000061,0.000003,0.000048,0.000002,0.000004
scheduler_factor,0.40947,0.182566,0.698639,0.343458,0.267422,0.784148,0.609429
scheduler_patience,5,5,3,10,6,8,9
threshold,0.058353,0.042006,0.053445,0.058376,0.055274,0.04666,0.048416
slope,6.459243,4.874079,8.644167,6.708292,9.415593,16.425836,10.775234
beta,0.830724,0.567498,0.788575,0.670362,0.888685,0.612527,0.681319
dropout_rate1,0.19794,0.503421,0.167363,0.386765,0.566866,0.888136,0.279804
dropout_rate2,0.734358,0.753545,0.169218,0.26509,0.344404,0.645687,0.189353
encoder_params,"PoissonEncoder(interval_freq=5, random_seed=47)",TBREncoder(threshold=1.373600005738427),StepForwardEncoder(threshold=0.21120508430205132),"BSAEncoder(win_size=8, cutoff=0.91192803486043...",PhaseEncoder(phase_window=1),"BurstEncoder(max_window=5, n_max=2, t_max=1, t...",DummyEncoder()


In [3]:
params_df.to_csv(REPORTS_DIR / 'encoder_params.csv')

NameError: name 'params_df' is not defined

In [ ]:
template = """
    \section{Poisson Encoder}
    \begin{table}[H]
    \centering
    \caption{Poisson Encoder: 5-Fold Cross-Validation Results with Total Spikes}
    \label{tab:poisson_kfold}
    \begin{tabular}{lccccc}
    \toprule
    \textbf{Fold} & \textbf{Accuracy} & \textbf{Precision} & \textbf{Recall} & \textbf{F1 Score} & \textbf{Total Spikes} \\
    \midrule
    Fold 1 & 77.59\% & 79.22\% & 76.30\% & 77.74\% & 45200 \\
    Fold 2 & 75.80\% & 72.15\% & 85.91\% & 78.39\% & 46110 \\
    Fold 3 & 75.99\% & 81.88\% & 65.78\% & 72.95\% & 45590 \\
    Fold 4 & 71.59\% & 72.80\% & 69.99\% & 71.36\% & 45950 \\
    Fold 5 & 73.18\% & 76.80\% & 62.88\% & 69.12\% & 46198 \\
    \midrule
    \textbf{Average} & 74.83\% & 76.57\% & 72.17\% & 73.91\% & 45609.74 \\
    \textbf{Std Dev} & 2.15\% & 3.72\% & 8.22\% & 3.61\% & 395.28 \\
    \bottomrule
    \end{tabular}
    \end{table}
"""

In [43]:
encoder_name = {
    "poisson": "Poisson Encoder",
    "tbr": "Threshould Based Representation",
    "sf": "Step Forward Encoder",
    "bsa": "Ben's Spiking Algorithm",
    "pe": "Phase Encoder",
    "be": "Burst Encoding",
    "dummy": "Dummy Encoder"
}

In [ ]:
import re

from tabulate import tabulate

for code in encoder_code:
    df = pd.read_csv(REPORTS_DIR / f'{code}_model_results_new.csv')
    df = df.drop(columns=['test_loss', 'test_mse'])
    df = df.rename(columns={'test_acc': 'Accuracy', 'test_f1': 'F1 Score', 'test_precision': 'Precision', 'test_recall': 'Recall', 'test_total_spikes': 'Average Spikes', 'fold': 'Fold'})
    df.set_index('Fold', inplace=True)
    df["Average Spikes"] = df["Average Spikes"].round(0)
    df.loc["Average"] = [df['Accuracy'].mean(), df['F1 Score'].mean(), df['Precision'].mean(), df['Recall'].mean(), df['Average Spikes'].mean()]
    df.loc["Std Dev"] = [df['Accuracy'].std(), df['F1 Score'].std(), df['Precision'].std(), df['Recall'].std(), df['Average Spikes'].std()]
    for col in ['Accuracy', 'F1 Score', 'Precision', 'Recall']:
        df[col] = df[col].apply(lambda x: f"{x:.2%}")
    df["Average Spikes"] = df["Average Spikes"].apply(lambda x: f"{x:.0f}")
    df.reset_index(inplace=True)
    df["Fold"] = df["Fold"].apply(lambda x: f"Fold {x + 1}" if x != "Average" and x != "Std Dev" else x)

    extract_table = re.compile(r"\\midrule([\s\S]*)\\bottomrule")
    raw_fold_part = tabulate(df.loc[:4], headers='keys', tablefmt='latex_booktabs', showindex=False)
    fold_part = extract_table.search(raw_fold_part).group(1).strip()

    bold_sum_part = re.compile(r"(Average|Std Dev)")

    raw_sum_part = tabulate(df.loc[5:], headers='keys', tablefmt='latex_booktabs', showindex=False)
    sum_part = extract_table.search(raw_sum_part).group(1).strip()
    sum_part = bold_sum_part.sub(r"\\textbf{\1}", sum_part)

    latex_output = r"""
\section{""" + encoder_name[code] + r"""}
\begin{table}[H]
\centering
\caption{""" + encoder_name[code] + r""": 5-Fold Cross-Validation Results with Total Spikes}
\label{tab:""" + code + r"""_kfold}
\begin{tabular}{lccccc}
\toprule
\textbf{Fold} & \textbf{Accuracy} & \textbf{Precision} & \textbf{Recall} & \textbf{F1 Score} & \textbf{Total Spikes} \\
\midrule
""" + fold_part + r"""
\midrule
""" + sum_part + r"""
\bottomrule
\end{tabular}
\end{table}
"""
    print(latex_output)


\section{Poisson Encoder}
\begin{table}[H]
\centering
\caption{Poisson Encoder: 5-Fold Cross-Validation Results with Total Spikes}
\label{tab:poisson_kfold}
\begin{tabular}{lccccc}
\toprule
\textbf{Fold} & \textbf{Accuracy} & \textbf{Precision} & \textbf{Recall} & \textbf{F1 Score} & \textbf{Total Spikes} \\
\midrule
Fold 1 & 77.59\%     & 79.22\%      & 76.30\%   & 77.74\%     &            45415 \\
 Fold 2 & 75.80\%     & 72.15\%      & 85.91\%   & 78.39\%     &            45992 \\
 Fold 3 & 76.00\%     & 81.88\%      & 65.78\%   & 72.95\%     &            45762 \\
 Fold 4 & 71.59\%     & 72.80\%      & 70.00\%   & 71.36\%     &            45574 \\
 Fold 5 & 73.18\%     & 76.80\%      & 62.88\%   & 69.12\%     &            45306 \\
\midrule
\textbf{Average} & 74.83\%     & 73.91\%      & 76.57\%   & 72.17\%     &            45610 \\
 \textbf{Std Dev} & 2.15\%      & 3.68\%       & 3.87\%    & 8.41\%      &              245 \\
\bottomrule
\end{tabular}
\end{table}


\section{Threshoul

: 

In [ ]:

print(sum_part)

\textbf{Average} & 77.02\%     & 77.81\%      & 75.94\%   & 80.32\%     &        181629    \\
 \textbf{Std Dev} & 2.62\%      & 1.82\%       & 5.55\%    & 4.06\%      &             0.06 \\
